# GEOCODING con GeoPy

    - Autor: afuentes
    - Fecha: 10/08/2021
    
## Objetivo

    - Trasnformar un set de direcciones a latitud y longitud.
    - Calcular distancia a un punto específico.

## 1 - Observamos la data

In [81]:
# Librerias
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
#import unidecode
import warnings
warnings.filterwarnings("ignore")

In [82]:
# importamos set de datos
df = pd.read_csv('direcciones.csv').drop(columns=['Unnamed: 0'])

In [83]:
# Vemos algunos registros de la data
df.head()

,Dirección,Comuna
0,Manuel Rodríguez 39,Santiago (Centro)
1,Sergio Valdovinos 1399,Santiago (Centro)
2,Huérfanos 1685,Santiago (Centro)
3,Av Fermín vivaceta 2040,Independencia
4,Maule 1467,Renca


In [84]:
# dimensión de la data
df.shape

(238, 2)

In [85]:
# datos nulos
df.isnull().sum()

Dirección     1
Comuna        1
dtype: int64

In [ ]:
# Borramos los datos NaN
df.dropna(inplace=True)

## 2.- Un poco de limpieza

En el nombre de las columnas quitamos los espacios y cambiamos minúsculas por mayúsculas.

In [86]:
# En los nombres de columnas cambiamos espacios por sin espacio y mayusculas las cambiamos por minusculas
df.columns = df.columns.str.replace(" ", "")
df.columns = df.columns.str.lower()

In [93]:
df.head()

,dirección,comuna,direccion_esp
0,Manuel Rodríguez 39,Santiago (Centro),"Manuel Rodríguez 39, Santiago , Chile"
1,Sergio Valdovinos 1399,Santiago (Centro),"Sergio Valdovinos 1399, Santiago , Chile"
2,Huérfanos 1685,Santiago (Centro),"Huérfanos 1685, Santiago , Chile"
3,Av Fermín vivaceta 2040,Independencia,"Av Fermín vivaceta 2040, Inpenncia, Chile"
4,Maule 1467,Renca,"Maule 1467, Renca, Chile"


## 3.- Preprocesamiento de dirección

Generamos una nueva columna la cual contenga la dirección de manera más específica. Esto lo logramos concatenando la columnas dirección, comuna y agregando el pais Chile.

In [89]:
df['direccion_esp'] = df['dirección'] + "," + " " + df['comuna'] + ","+ " " + 'Chile' 

Ademas quitamos algunos caracteres y palabras que nos pueden ensuciar la busqueda en la Api

In [91]:
df['direccion_esp'] = df['direccion_esp'].str.replace("(", "")
df['direccion_esp'] = df['direccion_esp'].str.replace(")", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("Centro", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("Calle", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("calle", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("de", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("del", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("De", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("Del", "")

In [92]:
# Observamos como queda nuestro df
df.head()

,dirección,comuna,direccion_esp
0,Manuel Rodríguez 39,Santiago (Centro),"Manuel Rodríguez 39, Santiago , Chile"
1,Sergio Valdovinos 1399,Santiago (Centro),"Sergio Valdovinos 1399, Santiago , Chile"
2,Huérfanos 1685,Santiago (Centro),"Huérfanos 1685, Santiago , Chile"
3,Av Fermín vivaceta 2040,Independencia,"Av Fermín vivaceta 2040, Inpenncia, Chile"
4,Maule 1467,Renca,"Maule 1467, Renca, Chile"


## 4.- Transformación de dirección a Latitud y Longitud

Ahora procedemos a "jugar" con la biblioteca GeoPy. Primero generamos nuestro objeto "geocoder". Aqui utilizamos el metodo "Nominatum".
Según la documentacion de GeoPy, para realizar las consultas a la API debe haber un retardo mínimo de un segundo. Para esto utilizamos la función "RateLimiter" la cual nos permitira realizar consultas con un 1s de delay entre consultas.

In [94]:
# Nominatim geocoder for OpenStreetMap data with RateLimiter
geocoder = RateLimiter(Nominatim(user_agent='MyApp2').geocode, min_delay_seconds=1)

Una vez instanciado nuestro elemento geocoder, prodecemos a entregarle nuestra data. A este objeto podemos solicitarle que nos entregue directamente la longitud y la latitud con .latitude o .longitude pero para esto habria que hacer dos llamados a la api y eso toma el doble de tiempo. Por eso opté por ocupar el metodo .raw para obtener toda la informacion de la direccion de una vez luego y despues filtrarla.

In [14]:
%%time
raw_list =[]

for idx, value in df2['direccion_esp'].iteritems():
    try:
        raw = geocoder(value).raw
        raw_list.append(raw)
    except:
        raw_list.append('No data')
        pass
    

Wall time: 3min 59s


In [97]:
# Vemos que es lo que guardo raw_list
raw_list[:2]

[{'place_id': 191661034,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 445535001,
  'boundingbox': ['-33.4565161', '-33.4557998', '-70.6574126', '-70.6570149'],
  'lat': '-33.456178',
  'lon': '-70.6571672',
  'display_name': 'Manuel Rodríguez, Santiago, Provincia de Santiago, Región Metropolitana de Santiago, 8370403, Chile',
  'class': 'highway',
  'type': 'primary',
  'importance': 0.5199999999999999},
 {'place_id': 110149353,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 89908536,
  'boundingbox': ['-33.4324663', '-33.4280714', '-70.707171', '-70.7064054'],
  'lat': '-33.4308179',
  'lon': '-70.7069057',
  'display_name': 'Sergio Valdovinos, Quinta Normal, Provincia de Santiago, Región Metropolitana de Santiago, WW, Chile',
  'class': 'highway',
  'type': 'secondary',
  'importance': 0.51}]

Observamos en que posición de la data obtenemos los datos que necesitamos

In [99]:
# Latitud
print(raw_list[0]['lat'])
# longitud
print(raw_list[0]['lon'])

-33.456178
-70.6571672


Procedemos a obtener la latitud y la longitud de toda la data.

In [38]:
lat = []
lon = []
for data in raw_list:  
    if data != 'No data':
        lat.append(data['lat'])
        lon.append(data['lon'])
    else:
        lat.append('No Data')
        lon.append('No Data')

Generamos dos series y la concatenamos al df

In [102]:
df['latitud'] = pd.Series(lat)
df['longitud'] = pd.Series(lon)

In [109]:
df.sample(5)

,dirección,comuna,direccion_esp,latitud,longitud
215,Buseta 4280,Cerrillos,"Buseta 4280, Cerrillos, Chile",No Data,No Data
104,Cuasimodo 3698,San Miguel,"Cuasimodo 3698, San Miguel, Chile",-33.48970374615385,-70.64257256923077
40,El roble 574,Recoleta,"El roble 574, Recoleta, Chile",-33.4049126,-70.6411061
17,Mar de chile 3694,Conchalí,"Mar chile 3694, Conchalí, Chile",-33.3970373,-70.6818167
99,Oscar Gonzáles 5421,Estación Central,"Oscar Gonzáles 5421, Estación Central, Chile",No Data,No Data


Verificamos cuantas direcciones NO se encontraron

In [111]:
df['latitud'].value_counts()

No Data                71
-33.3903291             8
-33.4494469             8
-33.4141433             6
-33.437259147619045     5
                       ..
-33.3626619             1
-33.455807750000005     1
-33.3867593             1
-33.4047774             1
-33.4062369             1
Name: latitud, Length: 104, dtype: int64

Podemos observar que 71 direcciones no fueron encontradas. El gran motivo es porque las direcciones vienen incompletas o bien, estan mal escritas. Me falta probar con la api de google. Creo que podria terner un mejor rendimiento pero este servicio es pagado :(.

## 5.- Calcular distancia a una dirección específica

Ahora procedemos a calcular la distancia a una direccion específica. Para esto le entregamos nuestra dirección especifica a geocoder para que la convierta a latitud y longitud. En este caso utilizaremos la dirección 'Dublé Almeyda 3143, Ñuñoa, Chile'.

In [112]:
coor_origen = (geocoder('Dublé Almeyda 3143, Ñuñoa, Chile').latitude, geocoder('Dublé Almeyda 3143, Ñuñoa, Chile').longitude)

In [113]:
coor_origen

(-33.4566638, -70.5984268)

Iteramos sobre las columnas del dataframe y calculamos la distancia con el método .distance donde se le entrega el par de coordeadas de destino y el par de cordenadas de origen.

In [115]:
distancia = []
for lat, lon in zip(df['latitud'], df['longitud']):
    if lat != 'No Data':
        d = distance.distance((lat, lon), (coor_origen[0], coor_origen[1])).km
        distancia.append(d)
    else:
        distancia.append('No Data')

Generamos una nueva columna con las distancia y la concatenamos a nuestro df

In [119]:
df['distancia'] = pd.Series(distancia)

In [120]:
df.sample(10)

,dirección,comuna,direccion_esp,latitud,longitud,distancia
56,General Freire 5011,Renca,"General Freire 5011, Renca, Chile",-33.4009058,-70.7242124,13.232
60,Rio quetro 6221,Estación Central,"Rio quetro 6221, Estación Central, Chile",-33.4689687,-70.7032165,9.83662
12,Av dorsol 1301,Conchalí,"Av dorsol 1301, Conchalí, Chile",No Data,No Data,No Data
52,Ismael Valdés 1954,Pedro Aguirre Cerda,"Ismael Valdés 1954, Pedro Aguirre Cerda, Chile",-33.4803263,-70.6569822,6.04277
130,Huérfanos 1919,Santiago (Centro),"Huérfanos 1919, Santiago , Chile",-33.44087445544655,-70.66390559348785,6.33491
157,Moneda 2180,Santiago (Centro),"Moneda 2180, Santiago , Chile",-33.443554006250004,-70.6658983875,6.43953
140,Milan 1426,San Miguel,"Milan 1426, San Miguel, Chile",-33.4775663,-70.6490781,5.24824
132,Marquesitas 3984,Renca,"Marquesitas 3984, Renca, Chile",No Data,No Data,No Data
53,Ismael Valdés 1954,Pedro Aguirre Cerda,"Ismael Valdés 1954, Pedro Aguirre Cerda, Chile",-33.4803263,-70.6569822,6.04277
179,Guardia marina 891,Santiago (Centro),"Guardia marina 891, Santiago , Chile",-33.5008539,-70.7504598,14.9566
